# Text Classification:

## Data

1. we have total of 20 types of documents(Text files) and total 18828 documents(text files).
2. You can download data from this <a href='https://drive.google.com/open?id=1rxD15nyeIPIAZ-J2VYPrDRZI66-TBWvM'>link</a>, in that you will get documents.rar folder. <br>If you unzip that, you will get total of 18828 documnets. document name is defined as'ClassLabel_DocumentNumberInThatLabel'. 
so from document name, you can extract the label for that document.
4. Now our problem is to classify all the documents into any one of the class.
5. Below we provided count plot of all the labels in our data. 

In [14]:
import tensorflow as tf
physical_device = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_device[0],True)
len(physical_device)


1

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model 
from tensorflow.keras.layers import Input,Dense,Conv1D,Flatten,Embedding,MaxPool1D,concatenate,Dropout
from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard,EarlyStopping
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings("ignore")

In [50]:

import glob
import os
import re
import numpy as np
import pandas as pd
import string
# !pip install tensorflow
# import tensorflow as tf


os.chdir("D:\\Applied_AI\\refrence for assignment 21\\documents")
files = glob.glob('*.txt')
print(len(files))

18828


In [17]:
def readlines(files):
    lst = []
    for fl in range(len(files)):
        raw = open(files[fl]).readlines()
        lst.append(raw)
    return lst
readlines_file = readlines(files)

In [18]:
def read(files):
    lst = []
    for fl in range(len(files)):
        raw = open(files[fl]).read()
        lst.append(raw)
    return lst
read_file = read(files)

In [19]:
label = []     #list contains Y_labels i.e text before'_'.
for i in range(len(files)):
    y = re.split('_',files[i])
    label.append(y[0])
len(label)

18828

In [20]:
# def preprocess0(files):    
#     preprocessed_emails = []    #Step 1
#     text_WO_mail = []    #Step 2

#     for i in read(files):
#         a = []
#         filtered = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',i)
#         modified = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',' ',i)
#         text_WO_mail.append(modified)
#         for j in filtered:
#             domain = j.split('@')[1].split('.')
#             for i in domain:
#                 if i == 'com':
#                     continue
#                 if len(i) > 2:
#                     a.append(i)
#         preprocessed_emails.append(' '.join(a))
#     return preprocessed_emails,text_WO_mail

In [21]:
def process_email(txt):
    emails = []
    a = []
    
    filtered = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',txt)
    for j in filtered:
        domain = j.split('@')[1].split('.')
        for i in domain:
            if i == 'com':
                continue
            if len(i) > 2:
                a.append(i)
    emails.append(' '.join(a))
    return emails.pop(0)

In [22]:
def process_sub(txt):
    for i in txt:
        if 'Subject:' in i:
            sub = i.split(':')[-1]
            sub1 = re.sub('\!|\"|\#|\$|\%|\&|\\|\'|\(|\)|\*|\+|\,|\-|\.|\/|\:|\;|\<|\=|\>|\?|\@|\[|\]|\^|\_|\`|\{|\}|\~|\n|\t','',sub)
    return sub1

In [23]:
def WO_mail(txt):
    return re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b',' ',txt)

In [24]:
def WO_sub(txt):
    return re.sub(r'Subject:[^\n]+',' ',txt)

In [25]:
def WO_From(txt):
    return re.sub(r'Write to:|From:[^\n]+','',txt)

In [26]:
def WO_tag(txt):
    return re.sub("<[^>]*>", "",txt)

In [27]:
def WO_bracket(txt):
    return re.sub(r'\([^)]*\)|(<.*>)',' ',txt)

In [28]:
def WO_Newlines(txt):
    return re.sub(r'\n|\t|\-|\\', '',txt)

In [29]:
def WO_colon(txt):
    return re.sub(r'[a-zA-Z]+:','',txt)

In [30]:
def decontract(txt):
    a = re.sub(r"won't", "will not", txt)
    a = re.sub(r"can\'t", "can not", txt)
    a = re.sub(r"n\'t", " not",txt)
    a = re.sub(r"\'re", " are", txt)
    a = re.sub(r"\'s", " is", txt)
    a = re.sub(r"\'d", " would", txt)
    a = re.sub(r"\'ll", " will", txt)
    a = re.sub(r"\'t", " not", txt)
    a = re.sub(r"\'ve", " have", txt)
    a = re.sub(r"\'m", " am", txt)
    
    return a

In [31]:
# import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')

In [32]:
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
def chunk(txt):
#     chunks = []
    chunks = (list(ne_chunk(pos_tag(word_tokenize(txt)))))   
    return chunks

In [33]:
# step 11 contd 
def combining_and_filtering(file, chunks):
    for i in chunks:
        if type(i)==Tree:
            if i.label() == "GPE":
                j = i.leaves()
                if len(j)>1:   #if new_delhi or bigger name
                    gpe = "_".join([term for term,pos in j])
                    f = re.sub(rf'{j[1][0]}',gpe,file, flags=re.MULTILINE)              #replacing delhi with new_delhi
                    file = re.sub(rf'\b{j[0][0]}\b',"",f, flags=re.MULTILINE)       #deleting new, \b is important
            if i.label() == "PERSON":           # deleting Ramesh         
                for term,pos in i.leaves():
                    file = re.sub(re.escape(term),"",file, flags=re.MULTILINE)

    return file


In [34]:
def WO_num(txt):
    return re.sub( r"[0-9]",' ',txt)

In [35]:
def WO_uScore(txt):
    return re.sub(r"\b_([a-zA-z]+)_\b|\b_([a-zA-z]+)\b|\b([a-zA-z]+)_\b",r"\1",txt)

In [36]:
def WO_smallwords(txt):
    return re.sub(r"\b[a-zA-Z]{1}_([a-zA-Z]+)|\b[a-zA-Z]{2}_([a-zA-Z]+)",r"\1",txt)

In [37]:
def lower_case(txt):
    file_lowr = txt.lower()
    a1 = re.sub(r'\b\w{1,2}\b'," ",file_lowr)
    aa = re.sub(r"\b\w{15,}\b"," ",a1)
    return aa

In [38]:
def WO_space(txt):
    txt_format = r"[^a-zA-Z_]"
    ab = re.sub(txt_format, " ",txt)
    aa = re.sub(r" {2,}", " ", ab, flags=re.MULTILINE)
    return aa

In [39]:
def preprocess(files):
    """Do all the Preprocessing as shown above and
    return a tuple contain preprocess_email,preprocess_subject,preprocess_text for that Text_data"""
    preprocess_text = []
    preprocessed_emails = []
    preprocess_subject = []
    for fl in range(len(files)):
        raw = open(files[fl]).read()
        raw_ = open(files[fl]).readlines()
        preprocessed_emails.append(process_email(raw))
#         for txt in files:
        a = WO_sub(raw)
        b = WO_From(a)
        c = WO_Newlines(b)
        d = WO_colon(c)
        e = decontract(d)
        f = chunk(e)
        g = combining_and_filtering(e,f)
        h = WO_num(g)
        i = WO_uScore(h)
        j = WO_smallwords(i)
        k = lower_case(j)
        l = WO_space(k)
        m = WO_mail(l)
        preprocess_text.append(m)
        preprocess_subject.append(process_sub(raw_))

    return preprocessed_emails,preprocess_text,preprocess_subject

In [40]:
p_mail,p_text,p_sub, = preprocess(['alt.atheism_49960.txt'])
p_text

[' atheism december atheist resources addresses atheist organizations usafreedom from religion foundation fish bumper stickers and assorted other atheist paraphernalia areavailable from the freedom from religion foundation the ffrf box evolution signs sell the fish fish symbol like the oness stick their cars but with feet and the word writteninside the deluxe moulded plastic fish postpaid the people the san francisco bay area can get from try mailing figmo netcom com for net people who directly theprice per fish american atheist pressaap publish various atheist books critiques the bible lists ofbiblical contradictions and one such book the bible handbook and american atheist isbn edition bible contradictions absurdities atrocities immoralities contains the itself aap based the king version the bible american atheist box austin austin prometheus bookssell books including see below east amherst street new_new_york alternate address which may newer older prometheus for humanisman organiza

In [41]:
p_mail, p_text, p_subject = preprocess(files)

In [42]:
len(p_mail),len(p_text),len(p_subject)

(18828, 18828, 18828)

In [43]:
data = {'preprocessed_mail':p_mail,'preprocessed_subject':p_subject,'preprocessed_text':p_text,'class':label}
df = pd.DataFrame(data)
df.head()
# X = pd.DataFrame(joined)
# y = label
# X

,preprocessed_mail,preprocessed_subject,preprocessed_text,class
0,mantis netcom mantis,Atheist Resources,atheism december atheist resources addresses ...,alt.atheism
1,mantis mantis mantis,Introduction to Atheism,atheism pril egin pgp signed messge introduct...,alt.atheism
2,dbstu1 tu-bs mimsy umd edu umd edu,Gospel Dating,article mimsy umd edu mangoe umd edu well has...,alt.atheism
3,mantis kepler unh edu,university violating separation of churchstate,dmn kepler unh edu until kings become philoso...,alt.atheism
4,Watson Ibm Com harder ccr-p ida org harder ccr...,socmotss et al Princeton axes matching funds ...,article apr harder ccrp ida org harder ccrp i...,alt.atheism


In [44]:
joined = []
for i in range(len(p_mail)):
    a = [p_subject[i],p_mail[i],p_text[i]]
    joined.append(" ".join(a))
# joined[100]

In [204]:
%load_ext tensorboard

In [45]:
X = pd.DataFrame(joined)
y = df['class']
X.shape,y.shape

((18828, 1), (18828,))

In [46]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
## encoding lables 
encoder = LabelEncoder()
encoder.fit(y)
encoder_y = encoder.transform(y)
## converting it to a matrix 
y = np_utils.to_categorical(encoder_y)

In [47]:
X.shape,y.shape

((18828, 1), (18828, 20))

In [48]:
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.25, stratify=y, random_state=82)
X_tr.shape, y_tr.shape, X_te.shape, y_te.shape

((14121, 1), (14121, 20), (4707, 1), (4707, 20))

In [51]:
sent_len = [len(i) for i in X_tr[0]]
sent_len.sort()
sent_len = np.array(sent_len)
sent_len

array([   15,    19,    21, ..., 43920, 46730, 54391])

In [52]:
percentiles = []
for i in range(97,101):
    a = int(np.percentile(sent_len,i))
    percentiles.append(a)
np.array(percentiles)

array([ 4455,  5694,  8942, 54391])

In [172]:
percentile_98 = int(np.percentile(sent_len,98))
percentile_981 = int(np.percentile(sent_len,98.1))
percentile_982 = int(np.percentile(sent_len,98.2))
percentile_983 = int(np.percentile(sent_len,98.3))
percentile_984 = int(np.percentile(sent_len,98.4))
percentile_985 = int(np.percentile(sent_len,98.5))
percentile_986 = int(np.percentile(sent_len,98.6))
percentile_987 = int(np.percentile(sent_len,98.7))
percentile_988 = int(np.percentile(sent_len,98.8))
percentile_989 = int(np.percentile(sent_len,96.9))


percentile_98,percentile_981,percentile_982,percentile_983,percentile_984,percentile_985,percentile_986,percentile_987,percentile_988,percentile_989,
percentile_989

4089

In [173]:
sent_len = [len(i) for i in X_te[0]]
sent_len.sort()
sent_len = np.array(sent_len)
sent_len

array([   11,    21,    21, ..., 38044, 40217, 47958])

In [174]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(
    num_words=None,
    filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n',
    lower=True,
    split=' ',
    char_level=False,
    oov_token=None,
    document_count=0,
)
tokenizer.fit_on_texts(X_tr[0])
X_tr1  = tokenizer.texts_to_sequences(X_tr[0])
X_te1 = tokenizer.texts_to_sequences(X_te[0])

In [175]:
X_tr2 = pad_sequences(X_tr1, maxlen=percentile_989, padding="post")
X_te2= pad_sequences(X_te1, maxlen=percentile_989, padding='post')

In [176]:
X_tr2.shape, X_te2.shape, y_tr.shape, y_te.shape

((14121, 4089), (4707, 4089), (14121, 20), (4707, 20))

In [177]:
os.chdir(r'D:\Applied_AI\refrence for assignment 21')

In [178]:
embedded_dict = {}
pretrain = open("glove.6B.50d.txt", encoding="utf8") 
for i in pretrain:
    value = i.split(" ")
    word = value[0]
    vector = np.asarray(value[1:])
    embedded_dict[word] = vector
pretrain.close()

In [179]:
## Converting embedding word to embedding matrix 
import numpy as np 

size = len(tokenizer.word_index)+1
emb_matrix = np.zeros((size, 50))
for word, a in tokenizer.word_index.items():
    emb_word = embedded_dict.get(word)

    if emb_word is not None:
      emb_matrix[a] = emb_word

### Model-1: Using 1D convolutions with word embeddings

In [180]:
## Embeding layer
embedding_layer = Embedding(len(tokenizer.word_index)+1, 50,
                            embeddings_initializer=tf.keras.initializers.Constant(emb_matrix),
                            trainable=False)

In [197]:
## Defining layer
first_layer = Input(shape=(percentile_989))


embed = embedding_layer(first_layer)
## hidden lyers
m1 = Conv1D(32,4,activation="relu",kernel_initializer =tf.keras.initializers.he_normal(),kernel_regularizer=tf.keras.regularizers.l2())(embed)
n1 = Conv1D(64,4,activation="relu",kernel_initializer =tf.keras.initializers.he_normal(),kernel_regularizer=tf.keras.regularizers.l2())(embed)
o1 = Conv1D(128,4,activation="relu",kernel_initializer = tf.keras.initializers.he_normal(),kernel_regularizer=tf.keras.regularizers.l2())(embed)

second_layer = concatenate([m1,n1,o1])

max_pool_1 = MaxPool1D(3)(second_layer)


i1 = Conv1D(32,3,activation="relu",kernel_initializer = tf.keras.initializers.he_normal(),kernel_regularizer=tf.keras.regularizers.l2())(max_pool_1)
j1 = Conv1D(32,3,activation="relu",kernel_initializer = tf.keras.initializers.he_normal(),kernel_regularizer=tf.keras.regularizers.l2())(max_pool_1)
k1 =  Conv1D(8,3,activation="relu",kernel_initializer = tf.keras.initializers.he_normal(),kernel_regularizer=tf.keras.regularizers.l2())(max_pool_1)


third_layer = concatenate([i1,j1,k1])
dropout_layer1 = Dropout(0.2)(third_layer)

max_pool_2 = MaxPool1D(3)(dropout_layer1)

##adding another layer
# i2 = Conv1D(128,3,activation="relu",kernel_initializer = tf.keras.initializers.he_normal(),kernel_regularizer=tf.keras.regularizers.l2())(max_pool_2)
# j2 = Conv1D(64,3,activation="relu",kernel_initializer = tf.keras.initializers.he_normal(),kernel_regularizer=tf.keras.regularizers.l2())(max_pool_2)
# k2 =  Conv1D(32,3,activation="relu",kernel_initializer = tf.keras.initializers.he_normal(),kernel_regularizer=tf.keras.regularizers.l2())(max_pool_2)

# fourth_layer = concatenate([i2,j2,k2])

# max_pool_3 = MaxPool1D(3)(fourth_layer)


# fifth_layer = Conv1D(32,3,activation='relu',kernel_initializer = tf.keras.initializers.he_normal(seed=42),kernel_regularizer=tf.keras.regularizers.l2())(max_pool_3)
fifth_layer = Conv1D(64,3,activation='relu',kernel_initializer = tf.keras.initializers.he_normal(seed=42),kernel_regularizer=tf.keras.regularizers.l2())(max_pool_2)

flatten = Flatten()(fifth_layer)

dropout_layer = Dropout(0.2)(flatten)

dense_layer = Dense(64,activation="relu",kernel_initializer = tf.keras.initializers.he_normal())(dropout_layer)
              
output_layer = Dense(20,activation="softmax",kernel_initializer= tf.keras.initializers.glorot_normal())(dense_layer)
              

model =Model(inputs=first_layer,outputs=output_layer) 

In [198]:
model.summary()

Model: "model_21"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           [(None, 4089)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 4089, 50)     7010050     input_22[0][0]                   
__________________________________________________________________________________________________
conv1d_147 (Conv1D)             (None, 4086, 32)     6432        embedding_2[3][0]                
__________________________________________________________________________________________________
conv1d_148 (Conv1D)             (None, 4086, 64)     12864       embedding_2[3][0]                
___________________________________________________________________________________________

In [199]:
# https://www.tensorflow.org/guide/keras/custom_callback
from sklearn.metrics import f1_score
class custom(tf.keras.callbacks.Callback):

  def on_train_begin(self,logs={}):
    self.f1_score_list = []

  def on_epoch_end(self,epoch,logs={}):

    x_val,y_val = X_te5, y_te

    pred_y = self.model.predict(x_val)

    y_t = np.zeros(y_val.shape[0])
    y_p = np.zeros(pred_y.shape[0])

    for i in range(len(y_t)):
      y_t[i] = int(np.argmax(y_val[i]))
      y_p[i] = int(np.argmax(y_p[i]))

    f1_value = f1_score(y_t,y_p,average="micro")
    print("f1_score:",f1_value)

    self.f1_score_list.append(f1_value)

In [200]:
from tensorflow_addons.metrics import F1Score
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping


## f1_score_callback and other callbacks
custom_callback = custom()
checkpoint = ModelCheckpoint(filepath='best_model_1.h5',verbose=2,monitor='val_accuracy', mode='max',save_best_only=True)
early_stop = EarlyStopping(monitor="val_accuracy",mode='max',patience=6)

## Tensorboard
log_dir = "logs"
tensorboard = TensorBoard(log_dir=log_dir,histogram_freq=1,write_graph=True)

## compiling the model 
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy', F1Score(average='micro',num_classes=20)])

In [202]:
# Trainning 
model.fit(X_tr2,y_tr,epochs=25,verbose=2,validation_data=(X_te2,y_te),batch_size =64,callbacks=[checkpoint,early_stop,tensorboard]) 

Epoch 1/25
221/221 - 32s - loss: 1.0093 - accuracy: 0.7771 - f1_score: 0.7771 - val_loss: 1.2932 - val_accuracy: 0.6871 - val_f1_score: 0.6871

Epoch 00001: val_accuracy improved from 0.68536 to 0.68706, saving model to best_model_1.h5
Epoch 2/25
221/221 - 30s - loss: 0.9743 - accuracy: 0.7887 - f1_score: 0.7887 - val_loss: 1.2651 - val_accuracy: 0.6953 - val_f1_score: 0.6953

Epoch 00002: val_accuracy improved from 0.68706 to 0.69535, saving model to best_model_1.h5
Epoch 3/25
221/221 - 30s - loss: 0.9595 - accuracy: 0.7942 - f1_score: 0.7942 - val_loss: 1.2651 - val_accuracy: 0.7032 - val_f1_score: 0.7032

Epoch 00003: val_accuracy improved from 0.69535 to 0.70321, saving model to best_model_1.h5
Epoch 4/25
221/221 - 30s - loss: 0.9501 - accuracy: 0.7985 - f1_score: 0.7985 - val_loss: 1.2923 - val_accuracy: 0.6975 - val_f1_score: 0.6975

Epoch 00004: val_accuracy did not improve from 0.70321
Epoch 5/25
221/221 - 31s - loss: 0.9544 - accuracy: 0.7992 - f1_score: 0.7992 - val_loss: 1.2

In [ ]:
%tensorboard --logdir logs

In [206]:
!rm -rf ./logs
# !kill 26912

'rm' is not recognized as an internal or external command,
operable program or batch file.


### Observation

## Model-2 : Using 1D convolutions with character embedding

In [207]:
X_tr3, X_te3, y_tr3, y_te3 = train_test_split(X, y, test_size=0.25, stratify=y)

In [208]:
X_tr3 = X_tr3[0]
X_te3 = X_te3[0]

In [209]:
tokenize_char = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n',char_level= True,oov_token='UNK')
tokenize_char.fit_on_texts(X_tr3)

In [210]:
print(tokenize_char.word_index)

{'UNK': 1, ' ': 2, 'e': 3, 't': 4, 'a': 5, 'o': 6, 'n': 7, 'i': 8, 's': 9, 'r': 10, 'h': 11, 'l': 12, 'd': 13, 'c': 14, 'u': 15, 'm': 16, 'p': 17, 'g': 18, 'y': 19, 'w': 20, 'f': 21, 'b': 22, 'v': 23, 'k': 24, 'x': 25, 'j': 26, 'q': 27, 'z': 28, '_': 29, '1': 30, '0': 31, '2': 32, '-': 33, '3': 34, '4': 35, '6': 36, "'": 37, '5': 38, '8': 39, '9': 40, '7': 41, '|': 42, '\\': 43, '\x08': 44, '\x10': 45}


In [211]:
size_char = len(tokenize_char.word_index)+1

In [212]:
## Tokenize them 
X_tr4 = tokenize_char.texts_to_sequences(X_tr3)
X_te4 = tokenize_char.texts_to_sequences(X_te3)

In [221]:
X_tr5 = pad_sequences(X_tr4, maxlen=percentile_989, padding="post")
X_te5 = pad_sequences(X_te4, maxlen=percentile_989, padding="post")
X_tr5.shape, X_te5.shape

((14121, 4089), (4707, 4089))

In [227]:
## Make a embedding matrix 
emb_matrix_char = np.zeros((46,61))
for i, j in tokenize_char.word_index.items():
  emb_matrix_char[j][j] = 1
print(emb_matrix_char)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [228]:
## embedding 
embedding_layer_char = Embedding(len(tokenize_char.word_index)+1,61, embeddings_initializer=tf.keras.initializers.Constant(emb_matrix_char),input_length=percentile_989,trainable=False)

In [229]:
## first layer
first_layer = Input(shape=(percentile_989))

embed = embedding_layer_char(first_layer)

In [230]:
m1 = Conv1D(64,3,activation="relu",kernel_initializer =tf.keras.initializers.he_normal(seed=42),kernel_regularizer=tf.keras.regularizers.l1())(embed)
n1 = Conv1D(64,3,activation="relu",kernel_initializer =tf.keras.initializers.he_normal(seed=42),kernel_regularizer=tf.keras.regularizers.l1())(m1)
max_pool_1 = MaxPool1D(5)(n1)

o1 = Conv1D(64,3,activation="relu",kernel_initializer = tf.keras.initializers.he_normal(seed=42),kernel_regularizer=tf.keras.regularizers.l1())(max_pool_1)
i1 = Conv1D(64,3,activation="relu",kernel_initializer = tf.keras.initializers.he_normal(seed=42),kernel_regularizer=tf.keras.regularizers.l1())(o1)


max_pool_2 = MaxPool1D(5)(i1)

i1 = Conv1D(64,3,activation="relu",kernel_initializer = tf.keras.initializers.he_normal(seed=42),kernel_regularizer=tf.keras.regularizers.l1())(max_pool_2)


max_pool_3 = MaxPool1D(5)(i1)

flatten = Flatten()(max_pool_3)

dropout_layer = Dropout(0.5)(flatten)

dense_layer1 = Dense(256,activation="relu",kernel_initializer = tf.keras.initializers.he_normal(seed=42))(dropout_layer)


              
output_layer = Dense(20,activation="softmax",kernel_initializer= tf.keras.initializers.glorot_normal(seed=42))(dense_layer1)
              

model2 = Model(inputs=first_layer,outputs=output_layer)

In [231]:
model2.summary()

Model: "model_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_28 (InputLayer)        [(None, 4089)]            0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 4089, 61)          2806      
_________________________________________________________________
conv1d_154 (Conv1D)          (None, 4087, 64)          11776     
_________________________________________________________________
conv1d_155 (Conv1D)          (None, 4085, 64)          12352     
_________________________________________________________________
max_pooling1d_44 (MaxPooling (None, 817, 64)           0         
_________________________________________________________________
conv1d_156 (Conv1D)          (None, 815, 64)           12352     
_________________________________________________________________
conv1d_157 (Conv1D)          (None, 813, 64)           123

In [232]:
## callbacks
f1_call = custom()
checkpoint = ModelCheckpoint(filepath='best_model_1.h5',verbose=1,monitor='val_accuracy', mode='auto',save_best_only=True)
early_stop = EarlyStopping(monitor="val_accuracy",mode='max',patience=2,verbose=1)

## Tensorboard
log_dir = "logs"
tensorboard = TensorBoard(log_dir=log_dir,histogram_freq=1,write_graph=True)
## reducing learning rate 

reduct = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy',patience=1,mode='auto',verbose=1,factor=0.9)

## compile model 
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy', F1Score(average='micro',num_classes=20)])


In [233]:
model.fit(X_tr5,y_tr3, epochs=10,validation_data=(X_te5, y_te3),batch_size =64,callbacks=[reduct , f1_call,checkpoint,early_stop,tensorboard]) 


Epoch 1/10
221/221 [==============================] - 34s 148ms/step - loss: 3.3339 - accuracy: 0.0797 - f1_score: 0.0797 - val_loss: 3.1538 - val_accuracy: 0.1160 - val_f1_score: 0.1160
f1_score: 0.04248990864669641

Epoch 00001: val_accuracy improved from -inf to 0.11600, saving model to best_model_1.h5
Epoch 2/10
221/221 [==============================] - 31s 138ms/step - loss: 3.0411 - accuracy: 0.1401 - f1_score: 0.1401 - val_loss: 2.9804 - val_accuracy: 0.1608 - val_f1_score: 0.1608
f1_score: 0.04248990864669641

Epoch 00002: val_accuracy improved from 0.11600 to 0.16082, saving model to best_model_1.h5
Epoch 3/10
221/221 [==============================] - 31s 139ms/step - loss: 2.8512 - accuracy: 0.1918 - f1_score: 0.1918 - val_loss: 2.7723 - val_accuracy: 0.2144 - val_f1_score: 0.2144
f1_score: 0.04248990864669641

Epoch 00003: val_accuracy improved from 0.16082 to 0.21436, saving model to best_model_1.h5
Epoch 4/10
221/221 [==============================] - 31s 138ms/step - lo

### Observations